In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
oil_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
transactions_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
holidays_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
stores_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

# Combine train_data and test_data.
combined_data = pd.concat([train_data, test_data], axis=0, ignore_index=True)

In [ ]:
# Function to summarize the data frames 
def resumetable(df):
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary['Missing Values'] = df.isna().sum().values
    summary['Unique Values'] = df.nunique().values
    summary['Min Value'] = df.min().values
    summary['Max Value'] = df.max().values
    return summary

def print_with_title(title, content):
    print("=" * 30)
    print(title)
    print("=" * 30)
    print(content)

train_summary = resumetable(train_data)
print_with_title("train_data summary", train_summary)

test_summary = resumetable(test_data)
print_with_title("test_data summary", test_summary)

holiday_summary = resumetable(holidays_data)
print_with_title("holidays_data summary", holiday_summary)

oil_summary = resumetable(oil_data)
print_with_title("oil_data summary", oil_summary)

transactions_summary = resumetable(transactions_data)
print_with_title("transactions_data summary", transactions_summary)

stores_summary = resumetable(stores_data)
print_with_title("stores_data summary", stores_summary)

In [ ]:
combined_data['date'] = pd.to_datetime(combined_data['date'])

# Create new features from date 
combined_data['year'] = combined_data['date'].dt.year  
combined_data['month'] = combined_data['date'].dt.month  
combined_data['day'] = combined_data['date'].dt.day
combined_data['day_of_week'] = combined_data['date'].dt.dayofweek
combined_data['is_weekend'] = combined_data['date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
combined_data['is_payday'] = combined_data['date'].dt.day.apply(lambda x: 1 if x in [15, 30] else 0)

combined_summary = resumetable(combined_data)
print_with_title("combined_data summary", combined_summary)

In [ ]:
# Function for payday variable 
def add_payday_flow(data):
    data['payday_flow'] = data['date'].dt.day.apply(
        lambda x: '30_before' if x >= 24 or x <= 1 else
                  '30_after' if 2 <= x <= 8 else
                  '15_before' if 9 <= x <= 14 else
                  '15_after' if 16 <= x <= 23 else 'other'
    )
    return data

combined_data = add_payday_flow(combined_data)

In [ ]:
# Function to process oil-related information. 
def preprocess_oil_data(oil_data):
    oil_data['date'] = pd.to_datetime(oil_data['date'])
    oil_data.set_index('date', inplace=True)
    oil_data['dcoilwtico'] = oil_data['dcoilwtico'].interpolate(method='time').bfill()
    oil_data.reset_index(inplace=True) 
    return oil_data.rename(columns={'dcoilwtico': 'oil_price'})

oil = preprocess_oil_data(oil_data)
print_with_title("Summary of oil_data", oil)

# Function to incorporate store information.
def preprocess_store_data(stores_data):
    stores_data.rename(columns={
        'city': 'store_city',
        'state': 'store_state',
        'type': 'store_type',
    }, inplace=True)
    return stores_data

stores = preprocess_store_data(stores_data)
print_with_title("Summary of stores_data stores_data", stores)

# Function to incorporate transaction information.
def preprocess_transactions_data(transactions_data):
    transactions_data['date'] = pd.to_datetime(transactions_data['date'])
    return transactions_data

transactions = preprocess_transactions_data(transactions_data)
print_with_title("Summary of transactions_data", transactions)

In [ ]:
def merge_features(combined_data, oil, transactions, stores):
  
    # Combine oil prices
    combined_data = combined_data.merge(oil, on='date', how='left')
    combined_data['oil_price'] = combined_data['oil_price'].fillna(method='ffill').fillna(method='bfill')

    # Combine transactions
    combined_data = combined_data.merge(transactions, on=['date', 'store_nbr'], how='left')
    combined_data['transactions'] = combined_data['transactions'].fillna(0)

    # Combine store characteristics 
    store_columns = ['store_nbr', 'store_city', 'store_state', 'store_type', 'cluster']
    combined_data = combined_data.merge(stores[store_columns], on='store_nbr', how='left')

    return combined_data
    
combined_data = merge_features(combined_data, oil, transactions, stores)
print(combined_data.columns)

In [ ]:
def preprocess_holiday_data(holidays_data, combined_data):
    holidays_data['date'] = pd.to_datetime(holidays_data['date'])
    holiday_data_filtered = holidays_data[
        (holidays_data['transferred'] != True) &  
        (holidays_data['type'] != "Work Day")    
    ].copy()

    holiday_data_filtered['is_holiday'] = np.where(
        holiday_data_filtered['type'] == "Holiday", "Yes", "No"
    )

    holiday_data_filtered['holiday_location'] = np.where(
        holiday_data_filtered['type'] == "Holiday",
        holiday_data_filtered['locale'],
        "Non-Holiday"
    )

    holiday_data_filtered['holiday_location_name'] = np.where(
        holiday_data_filtered['type'] == "Holiday",
        holiday_data_filtered['locale_name'],
        "Non-Holiday"
    )

    combined_data = combined_data.merge(
        holiday_data_filtered[['date', 'is_holiday', 'holiday_location', 'holiday_location_name']],
        on='date',
        how='left'
    )

    combined_data['is_holiday'] = combined_data['is_holiday'].fillna("No")
    combined_data['holiday_location'] = combined_data['holiday_location'].fillna("Non-Holiday")
    combined_data['holiday_location_name'] = combined_data['holiday_location_name'].fillna("Non-Holiday")

    # Create a holiday indicator for the store's location.
    def is_holiday_in_store_location(row):
        if row['is_holiday'] == "Yes" and row['holiday_location'] == "National":
            return "Yes"
        elif row['is_holiday'] == "Yes" and row['holiday_location'] == "Local" and row['store_city'] == row['holiday_location_name']:
            return "Yes"
        elif row['is_holiday'] == "Yes" and row['holiday_location'] == "Regional" and row['store_state'] == row['holiday_location_name']:
            return "Yes"
        else:
            return "No"

    combined_data['is_holiday_in_store_location'] = combined_data.apply(is_holiday_in_store_location, axis=1)

    combined_data['is_holiday_in_store_location'] = combined_data['is_holiday_in_store_location'].astype('category')

    # Remove unnecessary columns.
    columns_to_drop = ['date', 'holiday_location', 'holiday_location_name', 'type', 'description', 'transferred', 'locale', 'locale_name', 'is_holiday']
    combined_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    return combined_data

combined_data = preprocess_holiday_data(holidays_data, combined_data)
print(combined_data.dtypes)

To improve the accuracy of our predictions, we will scale some numerical columns, such as 'onpromotion' and 'oil_price'. The 'oil_price' column represents the price of oil on a specific day, while 'onpromotion' indicates the number of items on promotion on that day. These two metrics not only have different units but also vary in scale and range, which could impact the performance of the model.

By scaling these features, we ensure that their ranges are comparable, preventing any single feature from disproportionately influencing the model due to its magnitude. o ensure there is no data leakage, we will apply the scaling separately for the train and test datasets.

In [ ]:
# Split into train and test considering the sales variable
train = combined_data[~combined_data['sales'].isna()].copy()
test = combined_data[combined_data['sales'].isna()].copy()

# Select columns to scale
columns_to_scale = ['onpromotion', 'oil_price']

# Create the scaler
scaler = MinMaxScaler()

# Scale the values
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

print(train.head())
print(test.head())

In [ ]:
variables = ['year', 'month', 'day']

for var in variables:
    # Group by each unique value and calculate the average
    data_grouped = train.groupby(var)['sales'].mean().reset_index()
    
    # Plot grouped data
    plt.plot(data_grouped[var], data_grouped['sales'], marker='o')
    plt.title(f'Relationship between {var} and sales')
    plt.xlabel(var)
    plt.ylabel('Average sales')
    plt.grid(True)
    plt.show()


In [ ]:
variables = ['store_city', 'store_state', 'family', 'store_nbr'] 

for var in variables: 
    # Group by the current variable and calculate the average sales
    grouped_data = train.groupby(var)['sales'].mean().reset_index()

    # Create a bar chart
    plt.figure(figsize=(15, 8))
    sns.barplot(data=grouped_data, x=var, y='sales')
    plt.title(f'Promedio de ventas por {var}')
    plt.xlabel(var)
    plt.ylabel('Promedio de ventas')
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
columns = ['is_holiday_in_store_location', 'store_city', 'store_state', 'store_type', 'cluster', 'payday_flow', 'family']

def encode_categorical_columns(dataframe, columns):
    for col in columns:
        dataframe[col] = pd.Categorical(dataframe[col]).codes  
    return dataframe

train_v2 = encode_categorical_columns(train, columns)
test_v2 = encode_categorical_columns(test, columns)

correlaciones = train_v2.corr()  
correlation_target = correlaciones['sales'].sort_values(ascending=False)
print(correlation_target)

In [ ]:
# Combine train_data and test_data
combined_data_v2 = pd.concat([train_v2, test_v2], axis=0, ignore_index=True)

def calculate_family_averages(data):
    averages = data.groupby('family')[['sales', 'transactions']].mean().reset_index()
    averages.rename(columns={'sales': 'avg_sales', 'transactions': 'avg_transactions'}, inplace=True)
    return averages

# Calculate averages by family and add the new columns to the original DataFrame
family_averages = calculate_family_averages(combined_data_v2)
combined_data_v2 = combined_data_v2.merge(family_averages, on='family', how='left')

# Remove the 'transactions' column if it is no longer needed
combined_data_v2.drop(columns=['transactions'], inplace=True, errors='ignore')

train_v3 = combined_data_v2[~combined_data_v2['sales'].isna()].copy()
test_v3 = combined_data_v2[combined_data_v2['sales'].isna()].copy()

print(train_v3.head())

In [ ]:
# Transform sales to logarithmic scale
train_v3['sales_log'] = np.log1p(train_v3['sales'])  

# Separate independent and dependent variables
X = train_v3.drop(columns=['sales', 'sales_log'])  
y = train_v3['sales_log']  

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter selection for CatBoost
catboost_model = CatBoostRegressor(
    eval_metric='RMSE',
    verbose=False,
    random_seed=42
)

param_distributions = {
    'iterations': [500, 1000, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0.2, 0.5, 0.7, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=param_distributions,
    n_iter=10,                   
    scoring='neg_mean_squared_error',  
    cv=3,                        
    random_state=42,
    verbose=1
)


random_search.fit(X_train, y_train)

print("Best hyperparameters:", random_search.best_params_)


In [ ]:
# Train a CatBoost model with the previously obtained hyperparameters
catboost_model = CatBoostRegressor(
    iterations=1000,           
    learning_rate=0.1,        
    depth=8,                   
    eval_metric='RMSE',        
    verbose=False,             
    random_seed=42,
    l2_leaf_reg=5,
    bagging_temperature=0.7
)
catboost_model.fit(X_train, y_train)

# Make predictions on the logarithmic scale and transform them back to the original scale
catboost_predictions_log = catboost_model.predict(X_test)
catboost_predictions_original = np.expm1(catboost_predictions_log)  

# Transform y_test back to the original scale
y_test_original = np.expm1(y_test)

# Define the RMSLE metric and evaluate the model on the original scale
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

mae = mean_absolute_error(y_test_original, catboost_predictions_original)
mse = mean_squared_error(y_test_original, catboost_predictions_original)
rmse = np.sqrt(mse)  
r2 = r2_score(y_test_original, catboost_predictions_original)
rmsle_value = rmsle(y_test_original, catboost_predictions_original)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"RMSLE: {rmsle_value:.2f}")
print(f"R² Score: {r2:.2f}")

# Compare the first rows of predictions with actual values
output_df = pd.DataFrame({
    'Actual Sales (Original)': y_test_original,
    'Predicted Sales (Original)': catboost_predictions_original
})
print("\nFirst predictions of the model:")
print(output_df.head())

In [ ]:
# Make predictions on the test set
test_predictions_log = catboost_model.predict(test_v3) 

# Convert predictions back to the original scale
test_predictions = np.expm1(test_predictions_log)

# Create a DataFrame with the predictions and display the first rows 
output_catboost = pd.DataFrame({
    'sales': test_predictions
})
print(output_catboost.head())


In [ ]:
# Base model of LightGBM
lightgbm_model = LGBMRegressor(
    random_state=42,
    verbose=-1  
)

# Hyperparameter search space
param_distributions = {
    'n_estimators': [500, 1000, 2000],  
    'learning_rate': [0.01, 0.05, 0.1],  
    'max_depth': [4, 6, 8, 10],  
    'num_leaves': [20, 31, 50, 100],  
    'min_child_samples': [10, 20, 30],  
    'reg_alpha': [0, 0.1, 1, 10],  
    'reg_lambda': [0, 0.1, 1, 10],  
    'subsample': [0.6, 0.8, 1.0],  
    'colsample_bytree': [0.6, 0.8, 1.0]  
}

# RandomizedSearchCV for LightGBM
random_search = RandomizedSearchCV(
    estimator=lightgbm_model,
    param_distributions=param_distributions,
    n_iter=10,  
    scoring='neg_mean_squared_error',  
    cv=3,  
    random_state=42,
    verbose=1  
)

# Fit the model
random_search.fit(X_train, y_train)

print("Best hyperparameters:", random_search.best_params_)


In [ ]:
# Train a LightGBM model with the previously obtained hyperparameters
lightgbm_model = LGBMRegressor(
    n_estimators=1000,         
    learning_rate=0.05,        
    max_depth=8,               
    random_state=42,
    subsample=0.8, 
    reg_lambda=0.1, 
    reg_alpha=0, 
    num_leaves=100, 
    min_child_samples=20, 
    colsample_bytree=0.6,
    verbose=-1    
)
lightgbm_model.fit(X_train, y_train)

# Make predictions on the logarithmic scale and transform them back to the original scale
lightgbm_predictions_log = lightgbm_model.predict(X_test)
lightgbm_predictions_original = np.expm1(lightgbm_predictions_log)

# Transform y_test back to the original scale
y_test_original = np.expm1(y_test)

# Define the RMSLE metric and evaluate the model on the original scale
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

mae = mean_absolute_error(y_test_original, lightgbm_predictions_original)
mse = mean_squared_error(y_test_original, lightgbm_predictions_original)
rmse = np.sqrt(mse)  
r2 = r2_score(y_test_original, lightgbm_predictions_original)
rmsle_value = rmsle(y_test_original, lightgbm_predictions_original)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"RMSLE: {rmsle_value:.2f}")
print(f"R² Score: {r2:.2f}")

# Compare the first rows of predictions with actual values
output_df = pd.DataFrame({
    'Actual Sales (Original)': y_test_original,
    'Predicted Sales (Original)': lightgbm_predictions_original
})


In [ ]:
X_test_new = test_v3[X_train.columns]

# Make predictions on the test set
test_predictions_log = lightgbm_model.predict(X_test_new) 

# Convert predictions back to the original scale
test_predictions = np.expm1(test_predictions_log)

# Create a DataFrame with the predictions and display the first rows
output_lightgbm = pd.DataFrame({
    'sales': test_predictions
})
print(output_lightgbm.head())

In [ ]:
final_predictions = (output_catboost + output_lightgbm ) / 2

print(final_predictions.head())


In [ ]:
# Create the submission DataFrame with the columns 'id' and 'sales'
submission = test[['id']].copy()  
submission['sales'] = final_predictions['sales'].values  

print(submission.head())

# Save the CSV file
submission.to_csv('/kaggle/working/final_submission.csv', index=False)
print("Predictions file created.")
